In [1]:
import json, re

import pandas as pd
from langchain_openai import AzureChatOpenAI
from json.decoder import JSONDecodeError

In [2]:
df_scenarios = pd.read_csv("df_scenarios.csv")
df_claims = pd.read_csv("df_claims_with_document_path.csv")

In [3]:
df_claims = df_claims.drop(columns="reason")

df_scenarios = df_scenarios[['claim_type', 'covered_rules']]
df_scenarios = df_scenarios.drop_duplicates()

In [4]:
df_claims_original = df_claims.copy()

In [5]:
df_claims = df_claims.drop(columns=["document","document_path"])

In [6]:
creds_path = r"C:\Pranav\azure_openai_creds.json"

with open(creds_path, "r") as f:
    creds = json.load(f)

llm = AzureChatOpenAI(
    azure_endpoint=creds["azure_endpoint"],
    api_key=creds["OPENAI_API_KEY"],
    api_version=creds["OPENAI_API_VERSION"],
    deployment_name=creds["deployment_name"]
)

In [7]:
# df_scenarios["claim_type"].unique()

In [8]:
df_scenarios.head(2)

claim_type  \
0  Auto Insurance (Collision Coverage)   
1   Homeowners Insurance (HO-3 Policy)   

                                       covered_rules  
0  {'document_verification_agent': ['police_repor...  
1  {'document_verification_agent': ['proof_of_los...

In [9]:
# df_claims.head()

In [10]:
# for ind, row in df_scenarios.iterrows():

#     print(row["claim_type"])
#     print(row["covered_rules"])

In [11]:
df_claims.shape

(20, 8)

In [12]:
df_claims = df_claims.merge(df_scenarios[["claim_type","covered_rules"]], on = "claim_type", how="left")

In [13]:
df_claims.shape

(20, 9)

In [14]:
df_claims.head(2)

name policy_id policy_start  policy_end  \
0  Valid Vehicle  VHC-1001   2025-01-01  2025-12-31   
1  Valid Medical  HLT-2001   2025-01-01  2025-12-31   

                              claim_type accident_date  claim_amount  \
0    Auto Insurance (Collision Coverage)    2025-06-20          5000   
1  Health Insurance (Major Medical Plan)    2025-05-15         12000   

   is_valid_claim                                      covered_rules  
0            True  {'document_verification_agent': ['police_repor...  
1            True  {'document_verification_agent': ['submitted_do...

In [15]:
df_claims.columns

Index(['name', 'policy_id', 'policy_start', 'policy_end', 'claim_type',
       'accident_date', 'claim_amount', 'is_valid_claim', 'covered_rules'],
      dtype='object')

In [16]:
str(df_claims.to_dict("records")[0])

'{\'name\': \'Valid Vehicle\', \'policy_id\': \'VHC-1001\', \'policy_start\': \'2025-01-01\', \'policy_end\': \'2025-12-31\', \'claim_type\': \'Auto Insurance (Collision Coverage)\', \'accident_date\': \'2025-06-20\', \'claim_amount\': 5000, \'is_valid_claim\': True, \'covered_rules\': \'{\\\'document_verification_agent\\\': [\\\'police_report_validity: The police report must be complete and legible.\\\', \\\'photos_authenticity: Photographic evidence of the damage must be recent and consistent with the reported incident.\\\', \\\'repair_estimate_validity: Repair estimates must be provided with details from recognized service providers.\\\', "proof_of_insurance_validity: The submitted proof of insurance must match the policyholder\\\'s information and be current."], \\\'eligibility_checker_agent\\\': [\\\'policy_active_status: The policy must be active with all premiums paid up to date at the time of the incident.\\\', \\\'coverage_compliance: The damages claimed must fall under collis

In [17]:
for ind, row in df_claims.iterrows():

    # print(eval(row["covered_rules"]))
    for k, v in eval(row["covered_rules"]).items():
        print(k)
        print(v)
    break

document_verification_agent
['police_report_validity: The police report must be complete and legible.', 'photos_authenticity: Photographic evidence of the damage must be recent and consistent with the reported incident.', 'repair_estimate_validity: Repair estimates must be provided with details from recognized service providers.', "proof_of_insurance_validity: The submitted proof of insurance must match the policyholder's information and be current."]
eligibility_checker_agent
['policy_active_status: The policy must be active with all premiums paid up to date at the time of the incident.', 'coverage_compliance: The damages claimed must fall under collision coverage as outlined in the policy terms.', 'incident_date_verification: The incident must occur within the covered policy period.', 'driver_eligibility: The driver involved in the incident must be authorized and covered by the policy.']
fraud_checker_agent
['damage_consistency: The claimed damages must align with the police report a

In [46]:
df_claims.head(2)

name policy_id policy_start  policy_end  \
0  Valid Vehicle  VHC-1001   2025-01-01  2025-12-31   
1  Valid Medical  HLT-2001   2025-01-01  2025-12-31   

                              claim_type accident_date  claim_amount  \
0    Auto Insurance (Collision Coverage)    2025-06-20          5000   
1  Health Insurance (Major Medical Plan)    2025-05-15         12000   

   is_valid_claim                                      covered_rules  
0            True  {'document_verification_agent': ['police_repor...  
1            True  {'document_verification_agent': ['submitted_do...

In [54]:
df_claims.columns

Index(['name', 'policy_id', 'policy_start', 'policy_end', 'claim_type',
       'accident_date', 'claim_amount', 'is_valid_claim', 'covered_rules'],
      dtype='object')

In [49]:
# eval(df_claims["covered_rules"][0])

In [69]:
df_claims[df_claims["is_valid_claim"] == False].reset_index(drop=True).iloc[4,:]

name                                                   Policy Lapse
policy_id                                                  COM-6002
policy_start                                             2024-01-01
policy_end                                               2024-12-31
claim_type                            Commercial Property Insurance
accident_date                                            2025-01-08
claim_amount                                                  50000
is_valid_claim                                                False
covered_rules     {'document_verification_agent': ['submitted_do...
Name: 4, dtype: object

In [88]:
with open("claim_validation_prompt_new.txt", "r") as file:
    prompt = file.read()

row_index = 3

row = df_claims[df_claims["is_valid_claim"] == False].reset_index(drop=True).iloc[row_index,:]

input_data = prompt + "\n\nInput:\n" + row.to_json()
        
        # Call the LLM
response = llm.invoke(input_data)

# Log the raw response for debugging
# print(f"Raw LLM response for policy_id {row['policy_id']}: {response}")

# Extract content (response is an AIMessage object with .content as a string)
content = response.content if hasattr(response, 'content') else str(response)

print(eval(content))

['document_verification_agent - death_certificate_authenticity: The authenticity and accuracy of the death certificate may not have been verified, as assumed to have failed contributing to the invalid claim status.', 'document_verification_agent - policy_document_completeness: Necessary policy documents are assumed to have been complete and verified as passed despite the invalid claim status.', "document_verification_agent - beneficiary_identity: The claimant's identification may not have been valid or sufficiently verified, as assumed to have failed contributing to the invalid claim status.", 'eligibility_checker_agent - named_beneficiary_verification: The claimant is assumed not to be the named beneficiary, as assumed to have failed contributing to the invalid claim status.', "eligibility_checker_agent - incident_timeliness: The claim's submission timing is assumed to have aligned with policy requirements and thus passed despite the invalid claim status.", 'fraud_checker_agent - alte

In [89]:
eval(content)

['document_verification_agent - death_certificate_authenticity: The authenticity and accuracy of the death certificate may not have been verified, as assumed to have failed contributing to the invalid claim status.',
 'document_verification_agent - policy_document_completeness: Necessary policy documents are assumed to have been complete and verified as passed despite the invalid claim status.',
 "document_verification_agent - beneficiary_identity: The claimant's identification may not have been valid or sufficiently verified, as assumed to have failed contributing to the invalid claim status.",
 'eligibility_checker_agent - named_beneficiary_verification: The claimant is assumed not to be the named beneficiary, as assumed to have failed contributing to the invalid claim status.',
 "eligibility_checker_agent - incident_timeliness: The claim's submission timing is assumed to have aligned with policy requirements and thus passed despite the invalid claim status.",
 'fraud_checker_agent -

In [90]:
eval(df_claims["covered_rules"][row_index])

{'document_verification_agent': ['proof_of_loss_form: The proof of loss form must be provided and signed.',
  'incident_photos: Clear and relevant photos of the damage must be submitted.',
  'repair_estimate: The repair estimate must be included, itemized, and relevant to the reported damages.',
  'weather_report: A weather report must be provided for claims related to storm damage.',
  "plumber_report: A professional plumber's report is required for water damage claims caused by pipe issues.",
  'mitigation_proof: Evidence of mitigation efforts must be submitted when applicable.',
  'document_authenticity: All documents must appear authentic and free of tampering.'],
 'eligibility_checker_agent': ['policy_active_status: The policy must be active with premiums paid.',
  'damage_incident_date: The damage must have occurred during the active policy period.',
  'coverage_exclusions: The damage must not fall under policy exclusions (e.g., flood damage if flood coverage is excluded).',
  'i

In [ ]:
eval(df_claims["covered_rules"][row_index])

{'document_verification_agent': ['submitted_documents_completeness: All required documents, such as medical bills, medical records, and proof of payment, must be provided.',
  'authenticity_check: Submitted documents must be genuine and free from tampering or forgery.',
  'duplicate_document_check: Ensure that no duplicate bills or records are submitted for the same service.',
  'provider_details_verification: Verify that provider information (e.g., hospital or surgeon) on submitted documents matches official records.',
  'service_date_verification: Confirm that the service dates on documents are clearly stated and align with the claimed event.'],
 'eligibility_checker_agent': ['policy_active_status: The policy must be active with premiums paid at the time of the claimed service.',
  'coverage_scope_check: Verify that the claimed service is covered under the Major Medical Plan.',
  'network_status_verification: Confirm whether the provider is in-network for the policy, as this may impa

["document_verification_agent - police_report_validity: Police reports may not match the incident details, contributing to the claim's invalidity.",
 "fraud_checker_agent - incident_consistency_check: The incident details may not be consistent across all submitted documents, contributing to the claim's invalidity.",
 "fraud_checker_agent - exaggeration_detection: Claim amounts may be significantly inflated beyond repair estimates, contributing to the claim's invalidity.",
 "eligibility_checker_agent - compliance_with_policy_terms: The claim may not adhere to all conditions and exclusions outlined in the policy, contributing to the claim's invalidity.",
 "fraud_checker_agent - motive_analysis: Unusual circumstances may indicate potential fraudulent activity, contributing to the claim's invalidity."]

In [31]:
# df_claims = df_claims.drop(columns="notes")

In [21]:
# Load the prompt from the file
with open("claim_validation_prompt_new.txt", "r") as file:
    prompt = file.read()

# Initialize an empty list to store notes for each row
notes_list = []

# Iterate over each row in df_claims
for index, row in df_claims.iterrows():
    try:
        # Combine prompt with row data
        input_data = prompt + "\n\nInput:\n" + row.to_json()
        
        # Call the LLM
        response = llm.invoke(input_data)
        
        # Log the raw response for debugging
        print(f"Raw LLM response for policy_id {row['policy_id']}: {response}")
        
        # Extract content (response is an AIMessage object with .content as a string)
        content = response.content if hasattr(response, 'content') else str(response)
        
        # Log the content for debugging
        print(f"Response content for policy_id {row['policy_id']}: {content}")
        
        # Check for empty or None content
        if not content:
            print(f"No content for policy_id {row['policy_id']}")
            notes_list.append([])
            continue
        
        # Clean potential markdown wrapping
        cleaned_content = re.sub(r'^```json\n|\n```$', '', content.strip())
        
        # Try to parse the content as JSON
        try:
            notes = json.loads(cleaned_content)
        except JSONDecodeError as e:
            print(f"JSONDecodeError for policy_id {row['policy_id']}: {e}")
            print(f"Failed content: {cleaned_content}")
            notes = []  # Fallback to empty list
        
        # Validate that notes is a list of strings
        if not isinstance(notes, list) or not all(isinstance(note, str) for note in notes):
            print(f"Invalid response format for policy_id {row['policy_id']}: {cleaned_content}")
            notes = []  # Fallback to empty list
        
        # Append the notes to the list
        notes_list.append(notes)
    
    except Exception as e:
        print(f"Error for policy_id {row['policy_id']}: {e}")
        notes_list.append([])  # Fallback to empty list

# Add the notes column to df_claims
df_claims['notes'] = notes_list

Raw LLM response for policy_id VHC-1001: content='[\n  "document_verification_agent - police_report_validity: The police report is complete and legible, as inferred from the valid claim status.",\n  "document_verification_agent - photos_authenticity: Photographic evidence is recent and consistent, as inferred from the valid claim status.",\n  "document_verification_agent - repair_estimate_validity: Repair estimates are valid, as inferred from the valid claim status.",\n  "document_verification_agent - proof_of_insurance_validity: Proof of insurance is valid, as inferred from the valid claim status.",\n  "fraud_checker_agent - damage_consistency: Damages align with evidence, as inferred from the valid claim status.",\n  "fraud_checker_agent - repair_estimate_inconsistencies: No inconsistencies in repair estimates, as inferred from the valid claim status.",\n  "eligibility_checker_agent - driver_eligibility: The driver is authorized and covered by the policy, as inferred from the valid c

In [24]:
df_claims.head(2)

name policy_id policy_start  policy_end  \
0  Valid Vehicle  VHC-1001   2025-01-01  2025-12-31   
1  Valid Medical  HLT-2001   2025-01-01  2025-12-31   

                              claim_type accident_date  claim_amount  \
0    Auto Insurance (Collision Coverage)    2025-06-20          5000   
1  Health Insurance (Major Medical Plan)    2025-05-15         12000   

   is_valid_claim                                      covered_rules  \
0            True  {'document_verification_agent': ['police_repor...   
1            True  {'document_verification_agent': ['submitted_do...   

                                               notes  
0  [document_verification_agent - police_report_v...  
1  [document_verification_agent - submitted_docum...

In [26]:
# for ind, row in df_claims.iterrows():

#     # print(row["is_valid_claim"])
#     if row["is_valid_claim"] == False:
#         for k, v in eval(row["covered_rules"]).items():
#             print(k)
#             print("\n".join(v))
#             print("\n")

#         print("\n".join(row["notes"]))
#         break

In [27]:
df_claims.head(2)

name policy_id policy_start  policy_end  \
0  Valid Vehicle  VHC-1001   2025-01-01  2025-12-31   
1  Valid Medical  HLT-2001   2025-01-01  2025-12-31   

                              claim_type accident_date  claim_amount  \
0    Auto Insurance (Collision Coverage)    2025-06-20          5000   
1  Health Insurance (Major Medical Plan)    2025-05-15         12000   

   is_valid_claim                                      covered_rules  \
0            True  {'document_verification_agent': ['police_repor...   
1            True  {'document_verification_agent': ['submitted_do...   

                                               notes  
0  [document_verification_agent - police_report_v...  
1  [document_verification_agent - submitted_docum...

In [ ]:
for ind, row in df_claims.iterrows():

    print(row["is_valid_claim"])

In [186]:
df_claims.head(2)

name policy_id policy_start  policy_end  \
0  Valid Vehicle  VHC-1001   2025-01-01  2025-12-31   
1  Valid Medical  HLT-2001   2025-01-01  2025-12-31   

                              claim_type accident_date  claim_amount  \
0    Auto Insurance (Collision Coverage)    2025-06-20          5000   
1  Health Insurance (Major Medical Plan)    2025-05-15         12000   

   is_valid_claim                                      covered_rules  \
0            True  {'document_verification_agent': ['police_repor...   
1            True  {'document_verification_agent': ['submitted_do...   

                                               notes  
0  [document_verification_agent - police_report_v...  
1  [document_verification_agent - submitted_docum...

In [187]:
df_claims_original.head(2)

name policy_id policy_start  policy_end  \
0  Valid Vehicle  VHC-1001   2025-01-01  2025-12-31   
1  Valid Medical  HLT-2001   2025-01-01  2025-12-31   

                              claim_type accident_date  claim_amount  \
0    Auto Insurance (Collision Coverage)    2025-06-20          5000   
1  Health Insurance (Major Medical Plan)    2025-05-15         12000   

                         document  is_valid_claim  \
0         repair_bill_vehicle.pdf            True   
1  hospital_bill_knee_surgery.pdf            True   

                                       document_path  
0  C:\Pranav\claim_documents\repair_bill_vehicle.pdf  
1  C:\Pranav\claim_documents\hospital_bill_knee_s...

In [189]:
df_claims = df_claims.merge(df_claims_original[["policy_id","document","document_path"]], on = "policy_id", how="left")

In [191]:
df_claims.head(2)

name policy_id policy_start  policy_end  \
0  Valid Vehicle  VHC-1001   2025-01-01  2025-12-31   
1  Valid Medical  HLT-2001   2025-01-01  2025-12-31   

                              claim_type accident_date  claim_amount  \
0    Auto Insurance (Collision Coverage)    2025-06-20          5000   
1  Health Insurance (Major Medical Plan)    2025-05-15         12000   

   is_valid_claim                                      covered_rules  \
0            True  {'document_verification_agent': ['police_repor...   
1            True  {'document_verification_agent': ['submitted_do...   

                                               notes  \
0  [document_verification_agent - police_report_v...   
1  [document_verification_agent - submitted_docum...   

                         document  \
0         repair_bill_vehicle.pdf   
1  hospital_bill_knee_surgery.pdf   

                                       document_path  
0  C:\Pranav\claim_documents\repair_bill_vehicle.pdf  
1  C:\Pranav\claim_documents\hospital_bill_knee_s...

In [ ]:
# df_claims.to_csv("df_claims_with_notes.csv", index=False)